# CZII making datasets for YOLO

This is a challenging competition in which participants must identify the location of particles contained in a 3D volumetric image.

There are already some great baselines published, but most of them focus on 3D volumetric images.

However, using 3D images directly is difficult: for example, we always have to be careful about VRAM consumption: even a small 3D image uses a lot of memory.

Therefore, I propose to decompose the 3D data provided by the host into 2D image slices and reduce it to an object detection problem.

This method allows us to treat just 7 3D images as more than 1k 2D images, mitigating the data scarcity problem.

# Install and Import modules

In [4]:
# !pip install zarr

In [5]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zarr
from tqdm import tqdm
import glob, os
import cv2

In [6]:
runs = sorted(glob.glob('../../raw/train/overlay/ExperimentRuns/*'))
runs = [os.path.basename(x) for x in runs]
i2r_dict = {i:r for i, r in zip(range(len(runs)), runs)}
r2t_dict = {r:i for i, r in zip(range(len(runs)), runs)}
i2r_dict

{0: 'TS_5_4',
 1: 'TS_69_2',
 2: 'TS_6_4',
 3: 'TS_6_6',
 4: 'TS_73_6',
 5: 'TS_86_3',
 6: 'TS_99_9'}

# Normalize Function
To treat it as an image, normalize it to a value between 0 and 255.

1e-12 is very small and has the meaning of epsilon.

In [7]:
def convert_to_8bit(x):
    lower, upper = np.percentile(x, (0.5, 99.5))
    x = np.clip(x, lower, upper)
    x = (x - x.min()) / (x.max() - x.min() + 1e-12) * 255
    return x.round().astype("uint8")

# Information about labels

In [8]:
p2i_dict = {
        'apo-ferritin': 0,
        'beta-amylase': 1,
        'beta-galactosidase': 2,
        'ribosome': 3,
        'thyroglobulin': 4,
        'virus-like-particle': 5
    }

i2p = {v:k for k, v in p2i_dict.items()}

particle_radius = {
        'apo-ferritin': 60,
        'beta-amylase': 65,
        'beta-galactosidase': 90,
        'ribosome': 150,
        'thyroglobulin': 130,
        'virus-like-particle': 135,
    }

In [9]:
particle_names = ['apo-ferritin', 'beta-amylase', 'beta-galactosidase', 'ribosome', 'thyroglobulin', 'virus-like-particle']

In [10]:
json_each_paticle = f"../../raw/train/overlay/ExperimentRuns/TS_5_4/Picks/apo-ferritin.json"
df = pd.read_json(json_each_paticle) 
for axis in "x", "y", "z":
    df[axis] = df.points.apply(lambda x: x["location"][axis])
df.head(2)

,pickable_object_name,user_id,session_id,run_name,voxel_spacing,unit,points,trust_orientation,x,y,z
0,apo-ferritin,curation,0,TS_5_4,NaN,angstrom,"{'location': {'x': 468.514, 'y': 5915.906, 'z'...",True,468.514,5915.906,604.167
1,apo-ferritin,curation,0,TS_5_4,NaN,angstrom,"{'location': {'x': 5674.694, 'y': 1114.354, 'z...",True,5674.694,1114.354,565.068


# Main function for making datasets for YOLO
This is the main function.

Watch that YOLO annotation requires normalized 0 to 1 value range and (center_x, center_y, width, height) coordinate format.

In [11]:
def make_annotate_yolo(run_name, is_train_path=True):
    use_cols = ['exp_name', 'frame', 'x', 'y', 'height', 'width', 'label']
    df_result = pd.DataFrame(columns=use_cols)
    # to split validation
    is_train_path = 'train' if is_train_path else 'val'

    # read a volume
    vol = zarr.open(f'../../raw/train/static/ExperimentRuns/{r}/VoxelSpacing10.000/denoised.zarr', mode='r')
    # use largest images
    vol = vol[0]
    # normalize [0, 255]
    vol2 = convert_to_8bit(vol)
    
    n_imgs = vol2.shape[0]
    # process each slices
    for j in range(n_imgs):
        newvol = vol2[j]
        newvolf = np.stack([newvol]*3, axis=-1)
        # YOLO requires image_size is multiple of 32
        # 元画像のサイズを確認
        h, w = newvolf.shape[:2]

        # 必要なパディング量を計算 (左右と上下に均等に追加)
        padding = (640 - max(h, w)) // 2  # 10ピクセルのパディング

        # パディングを適用
        padded_image = cv2.copyMakeBorder(
            newvolf,
            top=padding,
            bottom=padding,
            left=padding,
            right=padding,
            borderType=cv2.BORDER_CONSTANT,
            value=(0, 0, 0)  # 黒背景（RGB画像の場合）
        )

        # 結果を確認
        # print(f"Original size: {h}x{w}")
        # print(f"Padded size: {padded_image.shape[:2]}")
        
        # newvolf = cv2.resize(newvolf, (640,640))
        # save as 1 slice
        cv2.imwrite(f'../../proc/yolo11_padding/images/{is_train_path}/{run_name}_{j*10}.png', padded_image)
        # make txt file for annotation
        with open(f'../../proc/yolo11_padding/labels/{is_train_path}/{run_name}_{j*10}.txt', 'w'):
            pass # make empty file
            
    # process each paticle types
    for p, particle in enumerate(tqdm(particle_names)):
        # we do not have to detect beta-amylase which weight is 0
        if particle=="beta-amylase":
            continue
        json_each_paticle = f"../../raw/train/overlay/ExperimentRuns/{run_name}/Picks/{particle}.json"
        df = pd.read_json(json_each_paticle) 
        # pick each coordinate of particles
        for axis in "x", "y", "z":
            df[axis] = df.points.apply(lambda x: x["location"][axis])

        
        radius = particle_radius[particle]
        for i, row in df.iterrows():
            # The radius from the center of the particle is used to determine the slices present.
            start_z = np.round(row['z'] - radius).astype(np.int32)
            start_z = max(0, start_z//10) # 10 means pixelspacing
            end_z = np.round(row['z'] + radius).astype(np.int32)
            end_z = min(n_imgs, end_z//10) # 10 means pixelspacing
            
            for j in range(start_z+1, end_z+1-1, 1):
                # 可視化用
                data = [
                    run_name, 
                    j*10, 
                    (row["x"]/10+padding)/640, 
                    (row["y"]/10+padding)/640, 
                    radius/10/640*2, 
                    radius/10/640*2, 
                    p2i_dict[particle]
                ]
                df_result.loc[len(df_result)] = data
                # white the results of annotation
                with open(f'../../proc/yolo11_padding/labels/{is_train_path}/{run_name}_{j*10}.txt', 'a') as f:
                    f.write(f'{p2i_dict[particle]} {(row["x"]/10+padding)/640} {(row["y"]/10+padding)/640} {radius/10/640*2} {radius/10/640*2} \n')
    
    return df_result

# Prepare Folders

In [12]:
os.makedirs("../../proc/yolo11_padding/images/train", exist_ok=True)
os.makedirs("../../proc/yolo11_padding/images/val", exist_ok=True)
os.makedirs("../../proc/yolo11_padding/labels/val", exist_ok=True)
os.makedirs("../../proc/yolo11_padding/labels/train", exist_ok=True)

# Main loop to make slice images and annotations

In [13]:
# {0: 'TS_5_4',
#  1: 'TS_69_2',
#  2: 'TS_6_4',
#  3: 'TS_6_6',
#  4: 'TS_73_6',
#  5: 'TS_86_3',
#  6: 'TS_99_9'}
list_df_result = []
for i, r in enumerate(runs):
    list_df_result.append(make_annotate_yolo(r, is_train_path=False if i==0 else True))

100%|██████████| 6/6 [00:05<00:00,  1.10it/s]


In [14]:
df_result = pd.concat(list_df_result).sort_values(by=["exp_name", "frame"]).reset_index(drop=True)
df_result

,exp_name,frame,x,y,height,width,label
0,TS_5_4,10,0.561035,0.297070,0.028125,0.028125,2
1,TS_5_4,10,0.803125,0.097418,0.040625,0.040625,4
2,TS_5_4,20,0.925042,0.809548,0.018750,0.018750,0
3,TS_5_4,20,0.561035,0.297070,0.028125,0.028125,2
4,TS_5_4,20,0.803125,0.097418,0.040625,0.040625,4
...,...,...,...,...,...,...,...
24794,TS_99_9,1430,0.241967,0.667303,0.040625,0.040625,4
24795,TS_99_9,1440,0.312185,0.850267,0.046875,0.046875,3
24796,TS_99_9,1440,0.145738,0.810348,0.046875,0.046875,3
24797,TS_99_9,1440,0.241967,0.667303,0.040625,0.040625,4


In [15]:
df_result.to_csv("df_labels_padding.csv", index=False)

Put them all in one folder.

In [16]:
import shutil
os.makedirs('../../proc/yolo11_padding/datasets/czii_det2d', exist_ok=True)
shutil.move('../../proc/yolo11_padding/images/train', '../../proc/yolo11_padding/datasets/czii_det2d/images/train')
shutil.move('../../proc/yolo11_padding/images/val', '../../proc/yolo11_padding/datasets/czii_det2d/images')
shutil.move('../../proc/yolo11_padding/labels/train', '../../proc/yolo11_padding/datasets/czii_det2d/labels/train')
shutil.move('../../proc/yolo11_padding/labels/val', '../../proc/yolo11_padding/datasets/czii_det2d/labels')

'../../proc/yolo11_padding/datasets/czii_det2d/labels/val'

# make yaml file for Training 
We need to create a yaml configuration file for training, the format of which will not be detailed here.

In [17]:
%%writefile ../../proc/yolo11_padding/czii_conf.yaml

path: /workspace/CZII/proc/yolo11_padding/datasets/czii_det2d # dataset root dir
train: images/train # train images (relative to 'path') 
val: images/val # val images (relative to 'path') 

# Classes
names:
  0: apo-ferritin
  1: beta-amylase
  2: beta-galactosidase
  3: ribosome
  4: thyroglobulin
  5: virus-like-particle

Writing ../../proc/yolo11_padding/czii_conf.yaml
